Transformación de nombres para adaptarlos de la lista descargada al link de internet:

In [1]:
import pandas as pd
import re
import unicodedata

df = pd.read_csv('Informe Listado de mesas.csv')

ListaEscuelas = df['nombre']

def eliminar_acentos(texto):
    # Normaliza el texto y elimina los caracteres diacríticos (acentos)
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )


def transformar_texto(texto):
    texto = texto.lower()  # Convertir todo a minúsculas
    texto = eliminar_acentos(texto) #Eliminar acentos
    texto = texto.replace(' ', '-')  # Reemplaza espacios por guiones
    texto = texto.replace('nº', 'n')  # Reemplaza 'Nº' por 'n'
    texto = re.sub(r'[^a-z0-9\-"]', '', texto)  # Elimina cualquier otro carácter especial
    texto = re.sub(r'-{2,}', '-', texto)  # Reemplaza múltiples guiones por uno solo
    return texto

ListaEscuelasWS = [transformar_texto(i) for i in ListaEscuelas]

ListaEscuelasWS

['esc-n-3-"del-centenario"',
 'esc-n-2-"domingo-f-sarmiento"',
 'esc-n-10-"r-de-escalada-de-san-martin"',
 'esc-n-402-"genaro-beron-de-astrada"',
 'colegio-polimodal-"ibera"',
 'esc-n-9-"gustavo-solano-gomez"',
 'esc-n-11-"dr-lisandro-segovia"',
 'esc-n-290-"dr-juan-r-vidal"',
 'esc-n-850-"gdor-jose-m-de-llano"',
 'esc-n-291-"dr-eudoro-vargas-gomez"',
 'esc-n-299-"juan-alfredo-asuad"',
 'colegio-"gral-san-martin"',
 'esc-n-4-"mariano-moreno"',
 'escuela-normal-"dr-juan-pujol"',
 'esc-n-1-"manuel-belgrano"',
 'esc-tecnica-n-2-"bernardino-rivadavia"',
 'esc-tecnica-n-1-"juana-manso"',
 'esc-n-607-"bernardino-rivadavia"',
 'esc-n-7-"isabel-e-vera"',
 'esc-n-275-"isidro-e-moreira"',
 'esc-n-146-"crucero-gral-belgrano"',
 'esc-n-345-"francisco-regis-ortiz"',
 'esc-comercial-"pte-dr-arturo-u-illia"',
 'esc-n-956-"n-sra-de-la-medalla-milagrosa"',
 'esc-n-139-"ambrosio-de-acosta"',
 'col-polim-"cautivas-correntinas"',
 'esc-n-368-"maestros-correntinos"',
 'col-sec-"fray-jose-de-la-quintana"',


Primera prueba para detectar los botones con Selenium:

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configura el driver de Chrome
service = Service('C:/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=service)

# Abre la página web
driver.get("https://elecciones2021.corrientes.gob.ar/escrutinio/escuela/escuela-normal-%22dr-juan-pujol%22")

try:
    # Esperar y encontrar el primer elemento (el icono)
    icono_elemento = WebDriverWait(driver, 15).until(
        EC.visibility_of_element_located((By.XPATH, '//span[@class="v-btn__content"]/i[@class="v-icon notranslate mdi mdi-format-list-bulleted-square theme--light"]'))
    )
    # Hacer clic en el primer elemento
    icono_elemento.click()

    # Verificar si el botón "CSV" está presente y visible
    boton_csv_presente = driver.find_elements(By.XPATH, '/html/body/div/div/div[1]/main/div/div/div[5]/div/div[1]/div[3]/div/div[1]/div/div[2]/button')
    if boton_csv_presente:
        print("El botón CSV está presente.")
    else:
        print("El botón CSV no se encontró.")  

except Exception as e:
    print("Ocurrió un error:", e)
    print(driver.page_source)  # Imprime el HTML actual para depuración

finally:
    driver.quit()

El botón CSV está presente.


Prueba para descargar uno de los archivos:

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

# Configurar opciones de Chrome para manejar descargas automáticamente
chrome_options = Options()
prefs = {
    "download.default_directory": "",  # Cambia a la ruta donde quieres que se guarden los archivos descargados
    "download.prompt_for_download": False,  # Deshabilita la ventana de diálogo de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False  # Deshabilita el chequeo de seguridad de Google para evitar bloqueos
}
chrome_options.add_experimental_option("prefs", prefs)

# Configura el driver de Chrome
service = Service('C:/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)

# Abre la página web
driver.get("https://elecciones2021.corrientes.gob.ar/escrutinio/escuela/escuela-normal-%22dr-juan-pujol%22")

try:
    # Esperar y encontrar el primer elemento (el icono)
    icono_elemento = WebDriverWait(driver, 5).until(
        EC.visibility_of_element_located((By.XPATH, '//span[@class="v-btn__content"]/i[@class="v-icon notranslate mdi mdi-format-list-bulleted-square theme--light"]'))
    )
    # Hacer clic en el primer elemento
    icono_elemento.click()

    # Esperar a que el botón CSV esté presente y visible
    boton_csv = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div[1]/main/div/div/div[5]/div/div[1]/div[3]/div/div[1]/div/div[2]/button'))
    )
    
    # Hacer clic en el botón CSV para iniciar la descarga
    boton_csv.click()

    print("Descarga iniciada.")

    # Esperar un tiempo para asegurarse de que la descarga se complete
    time.sleep(3)  # Ajusta el tiempo según la velocidad de tu conexión y el tamaño del archivo

except Exception as e:
    print("Ocurrió un error:", e)
    print(driver.page_source)  # Imprime el HTML actual para depuración

finally:
    driver.quit()

Descarga iniciada.


Descarga automatizada de todos los archivos:

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

# Configurar opciones de Chrome para manejar descargas automáticamente

escuelas_no_funcionan = []

for escuela in ListaEscuelasWS:
    # Abre la página web

    try:
        chrome_options = Options()

        prefs = {
            "download.default_directory": "",  # Cambia a la ruta donde quieres que se guarden los archivos descargados
            "download.prompt_for_download": False,  # Deshabilita la ventana de diálogo de descarga
            "download.directory_upgrade": True,
            "safebrowsing.enabled": False  # Deshabilita el chequeo de seguridad de Google para evitar bloqueos
        }
        chrome_options.add_experimental_option("prefs", prefs)

        # Configura el driver de Chrome
        service = Service('C:/chromedriver-win64/chromedriver.exe')

        driver = webdriver.Chrome(service=service, options=chrome_options)

        driver.get(f"https://elecciones2021.corrientes.gob.ar/escrutinio/escuela/{escuela}")

        try:
            # Esperar y encontrar el primer elemento (el icono)
            icono_elemento = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.XPATH, '//span[@class="v-btn__content"]/i[@class="v-icon notranslate mdi mdi-format-list-bulleted-square theme--light"]'))
            )
            # Hacer clic en el primer elemento
            icono_elemento.click()

            # Esperar a que el botón CSV esté presente y visible
            boton_csv = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div[1]/main/div/div/div[5]/div/div[1]/div[3]/div/div[1]/div/div[2]/button'))
            )
            
            # Hacer clic en el botón CSV para iniciar la descarga
            boton_csv.click()

            print("Descarga iniciada.")

            # Esperar un tiempo para asegurarse de que la descarga se complete
            time.sleep(3)  # Ajusta el tiempo según la velocidad de tu conexión y el tamaño del archivo

        except Exception as e:
            print("Ocurrió un error:", e)
            print(driver.page_source)  # Imprime el HTML actual para depuración

        finally:
            driver.quit()

    except Exception as e:
        # Si ocurre una excepción, agrega la escuela a la lista de errores
        print(f"Error al cargar la página de {escuela}.")
        escuelas_no_funcionan.append(escuela)

Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga iniciada.
Descarga ini